# Building a database for crime reports

In this project, we will build a database for storing data related with crimes that occured in Boston. The dataset is available in the file boston.csv .

The goal of this project is to create a database named crimes_db with a table – boston_crimes – with appropriate datatypes for storing the data from the boston.csv file. We will be creating the table inside a schema named crimes. We will also create the readonly and readwrite groups with the appropriate privileges. Finally, we will also need to create one user for each of these groups.

We will start by creating a database for storing our crime data as well as a schema for containing the tables.

In [ ]:
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cur = conn.cursor()
conn.autocommit = True
cur.execute("CREATE DATABASE crime_db;")
conn.autocommit = False
conn.close()

In [ ]:
conn = psycopg2.connect("dbname=crime_db user=dq")
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")
conn.commit()

We now have a database and a schema — we are ready to start creating tables. Before we do that, let's gather some data about our crime dataset so that we can more easily select the right datatypes to use in our table.

Let's start by reading the column names from the boston.csv file as well as the first row. In this way we will have them at hand throughout this guided project so that we can easily take a look at them at any moment.

In [3]:
import csv
with open('boston.csv','r') as file:
    reader = csv.reader(file)
    list_file = list(reader)
    col_headers = list_file[0]
    first_row = list_file[1]

We now have access to the column headers and the first row of data. This will help us when we create the table as we will not have to remember the names of the columns and using the first row of data we can easily recall what kind of data is stored in each column.

Before we create a table for storing the crime data, we need to identify the proper datatypes for the columns. To help us with that, let's create a function — get_col_value_set() — that given the name of a CSV file and a column index (starting a 0) that computes a Python set with all distinct values contained in that column.

This function will be useful for two reasons:
1. Checking whether an enumerated datatype might be a good choice for representing a column.
2. Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns.


In [4]:
def get_col_value_set(csv_filename, col_index):
    with open(csv_filename,'r') as file:
        reader = csv.reader(file)
        list_file = list(reader)
        rows = list_file[1:]
    
    value_set = set()
    for row in rows:
        value = row[col_index]
        if value not in value_set:
            value_set.add(value)
        else:
            continue
    return value_set

for i in range(len(col_headers)):
    value_set = get_col_value_set('boston.csv', i)
    max_length = 0
    print("There are " + str(len(value_set)) + " values in the set of column index " + str(i) + ".")
    for value in value_set:
        if len(value) > max_length:
            max_length = len(value)
    print("The maximum character length in this column is " + str(max_length) + ".")
    print("")

There are 298329 values in the set of column index 0.
The maximum character length in this column is 6.

There are 219 values in the set of column index 1.
The maximum character length in this column is 4.

There are 239 values in the set of column index 2.
The maximum character length in this column is 58.

There are 1177 values in the set of column index 3.
The maximum character length in this column is 10.

There are 7 values in the set of column index 4.
The maximum character length in this column is 9.

There are 18177 values in the set of column index 5.
The maximum character length in this column is 11.

There are 18177 values in the set of column index 6.
The maximum character length in this column is 12.



From the result above, we can analyze and select appropriate datatypes of each column.

Column index 4 (day_of_the_week) is an ideal candidate to have enumerated datatype.

Column index 2 (description) has the longest maximum length of string, so we have to keep note of that.

Let's create a table for storing the Boston crime data.

In [5]:
# identify suitable column names for the table, and first row to remember
# the kind of data contained in each column
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


- For column index 0, we will name incident_id with integer datatype is enough (4 bytes).
- For column index 1, we will name offense_id with smallint datatype (2 bytes)
- For column index 2, we will name description with varchar(100) datatype (with 42 spare length from maximum existing length).
- For column index 3, we will name date with date datatype (4 byte).
- For column index 4, we will name day with enumerated datatype (7 set values).
- For column index 5 and 6, we will name latitude and longitude with numeric(12,8) datatype.

In [8]:
# First create enumerated datatype for day column
cur.execute("""
CREATE TYPE daysweek AS ENUM('Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday');
""")

# Then, we can create the table as designed by our datatypes specification

cur.execute("""
CREATE TABLE crimes.boston_crimes (
    incident_id integer PRIMARY KEY,
    offense_id smallint,
    description varchar(100),
    date date,
    day daysweek,
    latitude numeric(12,8),
    longitude numeric(12,8)
);
""")
conn.commit()

ProgrammingError: type "daysweek" already exists


Now that we have created the table, we can load the data into it.

In [11]:
# Load the data from the boston.csv file into the crimes.boston_crimes table.
with open('boston.csv') as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", file)

IntegrityError: duplicate key value violates unique constraint "boston_crimes_pkey"
DETAIL:  Key (incident_id)=(1) already exists.
CONTEXT:  COPY boston_crimes, line 2


Now it is time to handle users. Our goal is to create the two user groups: readonly and readwrite. By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the public group and on the public schema.

In [14]:
# Revoke all the privileges of the public group on the public schema.
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")

# Revoke all privileges of public on the crime_db database.
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

conn.commit()

We have made sure that we are not going to inadvertently inherit privileges from the public group. The next step is to create our two users groups.

That the readonly group is supposed to only privileges to perform SELECT queries. In contrast, we want the readwrite group to be able to perform SELECT, INSERT, DELETE and UPDATE queries.

We also need:
- Connection privileges on the crime_db — otherwise, they won't be able to do anything.
- Usage of the crimes schema — this is where all data will be stored.

In [15]:
# Create two groups named readonly and readwrite with the no NOLOGIN option.
cur.execute("CREATE GROUP readonly;")
cur.execute("CREATE GROUP readwrite;")

# Grant CONNECT to the crime_db to both groups.
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")

# Grant USAGE to the crimes schema to both groups.
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

# Grant group specific privileges to each group on all tables in the crimes schema.
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

conn.commit()

We are nearly done with setting up our crime database. The only thing that we need to do is create users.

Let's wrap up the project by creating one user in each group. We will need to create each user and then assign them to each group.

In [16]:
# Create a user named data_analyst with password secret1.
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")

# Assign this user to the readonly group using the GRANT command.
cur.execute("GRANT readonly TO data_analyst;")

# Create a user named data_scientist with password secret2.
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")

# Assign this user to the readwrite group using the GRANT command.
cur.execute("GRANT readwrite TO data_scientist;")

conn.commit()

Postgres database has been set up.
It is a good practice to test that everything is configured as expected when we finish setting up the database.

In [19]:
# tests cell

cur.execute("SELECT * FROM pg_roles;")
result = cur.fetchall()
print(result)

cur.execute("""
SELECT grantee, privilege_type
FROM information_schema.table_privileges
WHERE grantee = 'readwrite';")

[('dq', True, True, True, True, True, True, True, -1, '********', None, None, 10), ('readonly', False, True, False, False, False, False, False, -1, '********', None, None, 16412), ('readwrite', False, True, False, False, False, False, False, -1, '********', None, None, 16413), ('data_analyst', False, True, False, False, False, True, False, -1, '********', None, None, 16414), ('data_scientist', False, True, False, False, False, True, False, -1, '********', None, None, 16415)]
